# Tabla de correlación diaria y mensual

In [1]:
'''Instalar Sklearn para usar mean_squared_error '''
# %pip install sklearn

'Instalar Sklearn para usar mean_squared_error '

In [2]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error,mean_absolute_percentage_error, r2_score,mean_absolute_error
from math import sqrt
from datetime import datetime
import os

In [3]:
from warnings import simplefilter
simplefilter(action="ignore", category=RuntimeWarning)
simplefilter(action="ignore", category=FutureWarning)

In [4]:
'''delimitar las fechas de inicio y fin para la correlacion diaria'''
fecha_ini = '1981-01-02'
fecha_fin = '2020-12-31'

'''nombre de las columnas'''
cols = ['fecha','100','101','102','105','106','107','108','111','112','113','114','115',
        '116','117','118','119','120','121','200','201','202','203','204','205']

'''tipos de correlacion'''
correl = ['spearman', 'pearson', 'kendall']


'''leer los archivos en un carpeta dada SI son archivos CSV'''
def leer_dir(path, cond = 'diario'):
    with os.scandir(path) as ficheros:
        ficheros = [fichero.name for fichero in ficheros if fichero.is_file() 
                                                            and fichero.name.endswith('.csv') 
                                                            and fichero.name.count(cond)>0]
    ficheros.sort()
    return ficheros

'''leer csv y retorna un DF con un rango de fechas entre <fecha_ini -  fecha_fin>'''
def leer_csv(archivo, ruta = '../datos/1.0.variables/'):
    p = ruta + archivo
    data = pd.read_csv(p, usecols = cols)[cols]
    data['fecha'] = pd.to_datetime(data['fecha'])
    data = data[(data['fecha']>=fecha_ini) & (data['fecha']<=fecha_fin)]
    data.reset_index(drop=True, inplace=True)
    return data

def creardf():
    data = pd.DataFrame(columns = cols)
    data = data.rename(columns = {'fecha':'info'})
    return data

def cadena(var, med1, med2, cor):
    v = var.split('.')
    m1 = med1.split('_')
    m2 = med2.split('_')
    return v[0]+'_'+m1[0]+'-'+m2[0]+'_'+cor

def correlacion(d0, d1, cor, columna):
    if cor=="kedall":
        data = d0.iloc[:,columna].corr(d1.iloc[:,columna], method = cor)
    else:
        data = d0.iloc[:,columna].corr(d1.iloc[:,columna], method = cor, min_periods=12)
    return data
    

## Correlacion diaria

In [5]:
carpeta = '../datos/1.0.variables/'
lista_d = leer_dir(carpeta, 'diario')

In [6]:
''' DF vacio para llenar datos,
    lista vacia para verificar si una variable se repite'''
datad = creardf()
realizado =[]
fila = 0


for archivo in lista_d:
    
    cad = archivo.split('_')
    
    '''condicion para verifica si la variable se repite'''
    if cad[2] in realizado:
        continue
    else:
        realizado.append(cad[2])
        '''buscar la variable en los nombres de los archivos para realizar la correlacion'''
        cad_match = [s for s in lista_d if cad[2] in s]
        if len(cad_match) == 3:
            d0 = leer_csv(cad_match[0])
            d1 = leer_csv(cad_match[1])
            d2 = leer_csv(cad_match[2])
            
            datad.loc[fila] = d0.isnull().sum()
            datad.iloc[fila,0] = 'nulos_'+cad_match[0]
            datad.loc[fila+1] = d1.isnull().sum()
            datad.iloc[fila+1,0] = 'nulos_'+cad_match[1]
            datad.loc[fila+2] = d2.isnull().sum()
            datad.iloc[fila+2,0] = 'nulos_'+cad_match[2]
            
            fila = fila + 3
            
            for columna in range(d0.shape[1]):
                if columna == 0:
                    datad.loc[fila,'info'] = cadena(cad[2], cad_match[0], cad_match[1], correl[0])
                    datad.loc[fila+1,'info'] = cadena(cad[2], cad_match[0], cad_match[2], correl[0])
                    datad.loc[fila+2,'info'] = cadena(cad[2], cad_match[2], cad_match[1], correl[0])
                    datad.loc[fila+3,'info'] = cadena(cad[2], cad_match[0], cad_match[1], correl[1])
                    datad.loc[fila+4,'info'] = cadena(cad[2], cad_match[0], cad_match[2], correl[1])
                    datad.loc[fila+5,'info'] = cadena(cad[2], cad_match[2], cad_match[1], correl[1])
                    datad.loc[fila+6,'info'] = cadena(cad[2], cad_match[0], cad_match[1], correl[2])
                    datad.loc[fila+7,'info'] = cadena(cad[2], cad_match[0], cad_match[2], correl[2])
                    datad.loc[fila+8,'info'] = cadena(cad[2], cad_match[2], cad_match[1], correl[2])
                else:
                    datad.iloc[fila, columna] = correlacion(d0, d1, correl[0], columna)
                    datad.iloc[fila+1, columna] = correlacion(d0, d2, correl[0], columna)
                    datad.iloc[fila+2, columna] = correlacion(d2, d1, correl[0], columna)
                    datad.iloc[fila+3, columna] = correlacion(d0, d1, correl[1], columna)
                    datad.iloc[fila+4, columna] = correlacion(d0, d2, correl[1], columna)
                    datad.iloc[fila+5, columna] = correlacion(d2, d1, correl[1], columna)
                    datad.iloc[fila+6, columna] = correlacion(d0, d1, correl[2], columna)
                    datad.iloc[fila+7, columna] = correlacion(d0, d2, correl[2], columna)
                    datad.iloc[fila+8, columna] = correlacion(d2, d1, correl[2], columna)
            fila = fila+9
        elif len(cad_match) == 2:
            d0 = leer_csv(cad_match[0])
            d1 = leer_csv(cad_match[1])
            
                        
            '''calcular los nullos en cada columna y en cada archivo'''
            datad.loc[fila] = d0.isnull().sum()
            datad.iloc[fila,0] = 'nulos_'+cad_match[0]
            datad.loc[fila+1] = d1.isnull().sum()
            datad.iloc[fila+1,0] = 'nulos_'+cad_match[1]
            
            fila = fila + 2
            
            
            for columna in range(d0.shape[1]):
                if columna == 0:
                    datad.loc[fila,'info'] = cadena(cad[2], cad_match[0], cad_match[1], correl[0])
                    datad.loc[fila+1,'info'] = cadena(cad[2], cad_match[0], cad_match[1], correl[1])
                    datad.loc[fila+2,'info'] = cadena(cad[2], cad_match[0], cad_match[1], correl[2])
                else:
                    datad.iloc[fila,columna] = correlacion(d0, d1, correl[0], columna)
                    datad.iloc[fila+1,columna] = correlacion(d0, d1, correl[1], columna)
                    datad.iloc[fila+2,columna] = correlacion(d0, d1, correl[2], columna)
            fila = fila + 3
datad.to_csv('../datos/2.3.Resumen/resumen_correlacion_diaria.csv')

## Correlación mensual

In [7]:
carpeta = '../datos/1.0.variables/'
lista_m = leer_dir(carpeta, 'mensual')


In [13]:
''' DF vacio para llenar datos,
    lista vacia para verificar si una variable se repite'''
datam = creardf()
realizado =[]
fila = 0


for archivo in lista_m:
    
    cad = archivo.split('_')
    print(cad[2])
    '''condicion para verifica si la variable se repite'''
    if cad[2] in realizado:
        continue
    else:
        realizado.append(cad[2])
        '''buscar la variable en los nombres de los archivos para realizar la correlacion'''
        cad_match = [s for s in lista_m if cad[2] in s]
        print(cad_match)
        if len(cad_match) == 3:
            d0 = leer_csv(cad_match[0])
            d1 = leer_csv(cad_match[1])
            d2 = leer_csv(cad_match[2])
            
            
            '''calcular los nullos en cada columna y en cada archivo'''
            datam.loc[fila] = d0.isnull().sum()
            datam.iloc[fila,0] = 'nulos_'+cad_match[0]
            datam.loc[fila+1] = d1.isnull().sum()
            datam.iloc[fila+1,0] = 'nulos_'+cad_match[1]
            datam.loc[fila+2] = d2.isnull().sum()
            datam.iloc[fila+2,0] = 'nulos_'+cad_match[2]
            
            fila = fila + 3
            
            
            for columna in range(d0.shape[1]):
                if columna == 0:
                    datam.loc[fila,'info'] = cadena(cad[2], cad_match[0], cad_match[1], correl[0])
                    datam.loc[fila+1,'info'] = cadena(cad[2], cad_match[0], cad_match[2], correl[0])
                    datam.loc[fila+2,'info'] = cadena(cad[2], cad_match[2], cad_match[1], correl[0])
                    datam.loc[fila+3,'info'] = cadena(cad[2], cad_match[0], cad_match[1], correl[1])
                    datam.loc[fila+4,'info'] = cadena(cad[2], cad_match[0], cad_match[2], correl[1])
                    datam.loc[fila+5,'info'] = cadena(cad[2], cad_match[2], cad_match[1], correl[1])
                    datam.loc[fila+6,'info'] = cadena(cad[2], cad_match[0], cad_match[1], correl[2])
                    datam.loc[fila+7,'info'] = cadena(cad[2], cad_match[0], cad_match[2], correl[2])
                    datam.loc[fila+8,'info'] = cadena(cad[2], cad_match[2], cad_match[1], correl[2])
                else:
                    datam.iloc[fila, columna] = correlacion(d0, d1, correl[0], columna)
                    datam.iloc[fila+1, columna] = correlacion(d0, d2, correl[0], columna)
                    datam.iloc[fila+2, columna] = correlacion(d2, d1, correl[0], columna)
                    datam.iloc[fila+3, columna] = correlacion(d0, d1, correl[1], columna)
                    datam.iloc[fila+4, columna] = correlacion(d0, d2, correl[1], columna)
                    datam.iloc[fila+5, columna] = correlacion(d2, d1, correl[1], columna)
                    datam.iloc[fila+6, columna] = correlacion(d0, d1, correl[2], columna)
                    datam.iloc[fila+7, columna] = correlacion(d0, d2, correl[2], columna)
                    datam.iloc[fila+8, columna] = correlacion(d2, d1, correl[2], columna)
            fila = fila+9
        elif len(cad_match) == 2:
            d0 = leer_csv(cad_match[0])
            d1 = leer_csv(cad_match[1])
            
            
            '''calcular los nullos en cada columna y en cada archivo'''
            datam.loc[fila] = d0.isnull().sum()
            datam.iloc[fila,0] = 'nulos_'+cad_match[0]
            datam.loc[fila+1] = d1.isnull().sum()
            datam.iloc[fila+1,0] = 'nulos_'+cad_match[1]
            
            fila = fila + 2
            
            
            for columna in range(d0.shape[1]):
                if columna == 0:
                    datam.loc[fila,'info'] = cadena(cad[2], cad_match[0], cad_match[1], correl[0])
                    datam.loc[fila+1,'info'] = cadena(cad[2], cad_match[0], cad_match[1], correl[1])
                    datam.loc[fila+2,'info'] = cadena(cad[2], cad_match[0], cad_match[1], correl[2])
                else:
                    datam.iloc[fila,columna] = correlacion(d0, d1, correl[0], columna)
                    datam.iloc[fila+1,columna] = correlacion(d0, d1, correl[1], columna)
                    datam.iloc[fila+2,columna] = correlacion(d0, d1, correl[2], columna)
            fila = fila + 3
datam.to_csv('../datos/2.3.Resumen/resumen_correlacion_mensual.csv')

pp.csv
['chirps_mensual_pp.csv', 'nasa_mensual_pp.csv', 'obs_mensual_pp.csv']
hmed.csv
['nasa_mensual_hmed.csv', 'obs_mensual_hmed.csv']
pp.csv
tmax.csv
['nasa_mensual_tmax.csv', 'obs_mensual_tmax.csv']
tmed.csv
['nasa_mensual_tmed.csv', 'obs_mensual_tmed.csv']
tmin.csv
['nasa_mensual_tmin.csv', 'obs_mensual_tmin.csv']
oni.csv
['noaa_mensual_oni.csv']
hmed.csv
pp.csv
tmax.csv
tmed.csv
tmin.csv


In [9]:
datam

,info,100,101,102,105,106,107,108,111,112,...,118,119,120,121,200,201,202,203,204,205
0,nulos_chirps_mensual_pp.csv,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,nulos_nasa_mensual_pp.csv,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,nulos_obs_mensual_pp.csv,0.000000,154.000000,402.000000,188.000000,317.000000,413.000000,170.000000,338.000000,126.000000,...,359.000000,200.000000,236.000000,390.000000,27.000000,2.000000,15.000000,2.000000,176.000000,26.000000
3,pp_chirps-nasa_spearman,0.919411,0.904852,0.900617,0.906874,0.891426,0.905022,0.825536,0.839398,0.895416,...,0.841415,0.881909,0.878368,0.879062,0.891042,0.897776,0.877304,0.899417,0.875278,0.891485
4,pp_chirps-obs_spearman,0.911229,0.746139,0.872861,0.868672,0.805443,0.651895,0.767873,0.751391,0.814323,...,0.816857,0.756051,0.840893,0.590864,0.848153,0.835782,0.822635,0.874328,0.805161,0.802270
5,pp_obs-nasa_spearman,0.959863,0.754064,0.900057,0.869717,0.767769,0.673653,0.753099,0.688209,0.782753,...,0.839939,0.753797,0.878177,0.622690,0.833591,0.868625,0.781984,0.858410,0.802368,0.859485
6,pp_chirps-nasa_pearson,0.651042,0.842365,0.672158,0.636459,0.872044,0.804247,0.805279,0.817076,0.862473,...,0.837302,0.822011,0.743346,0.868742,0.832081,0.877348,0.818756,0.873832,0.843938,0.824711
7,pp_chirps-obs_pearson,0.908940,0.783326,0.920683,0.854710,0.811628,0.779192,0.746802,0.669612,0.772379,...,0.832124,0.833204,0.846556,0.406263,0.882002,0.852197,0.665396,0.884005,0.803714,0.822043
8,pp_obs-nasa_pearson,0.683810,0.695182,0.869713,0.531546,0.707859,0.721742,0.676743,0.601111,0.714158,...,0.756614,0.663909,0.803670,0.524386,0.797050,0.805155,0.543763,0.826164,0.666439,0.768788
9,pp_chirps-nasa_kendall,0.768516,0.744671,0.737378,0.745539,0.733153,0.744431,0.642547,0.660970,0.738086,...,0.666403,0.707316,0.703875,0.715569,0.723922,0.731307,0.702173,0.743662,0.695818,0.727387
